# PROYECTO FINAL: NullStorm

## 1. MODELO COMPUTER VISION

* Google Drive

In [1]:
from google.colab import drive
from google.colab import auth
auth.authenticate_user()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* Librerías necesarias

In [2]:
from google.colab import files
from google.cloud import bigquery
import torch #Biblioteca para trabajar con modelos de aprendizaje profundo
import cv2 # Biblioteca para procesamiento de imágenes y video
import time # Para medir tiempos de ejecución
import re # Biblioteca para trabajar con expresiones regulares
import numpy as np # Biblioteca para cálculos numéricos
import easyocr # Biblioteca OCR para reconocimiento de texto
import os # Biblioteca para manejo de archivos y directorios
from IPython.display import Image, display # Para mostrar imágenes en Jupyter Notebook
import yolov5 # Biblioteca YOLOv5 para detección de objetos
import matplotlib.pyplot as plt
import re
import random
from datetime import datetime, timedelta

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 1.1. Función para cargar la imagen

In [3]:
def img_load(img_path=None):

    if img_path is None:
        print("Por favor, selecciona una imagen del vehículo:")
        uploaded = files.upload()
        image_path = next(iter(uploaded))
        frame = cv2.imread(image_path)
        # Convertir la imagen de BGR a RGB(Asegurar la compatibilidad entre las diferentes partes del pipeline de procesamiento de imágenes)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame

    else:
        frame = cv2.imread(img_path) # Lee la imagen
        # Convertir la imagen de BGR a RGB(Asegurar la compatibilidad entre las diferentes partes del pipeline de procesamiento de imágenes)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame


## 1.2. Detección de objetos con YOLOv5

* Se cargan los modelos YOLOv5 para detección de vehículos y YOLOv5 para detección de matrículas.

In [4]:
vehicle_model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # Modelo YOLOv5 para detección de vehículos
plate_model = yolov5.load('keremberke/yolov5m-license-plate') # Modelo YOLOv5 para detección de placas

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-2-21 Python-3.11.11 torch-2.5.1+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 133MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/42.1M [00:00<?, ?B/s]

* Función para la detección de objetos

In [29]:
def detectx(frame, model):
    #Args: frame(Imagen en la que se realizarán las detecciones), model(Modelo YOLOv5 a utilizar)
    results = model(frame)

    # Convertir a lista y acceder al primer objeto detectado
    detections = results.xyxy[0]  # tensor con las detecciones

    if len(detections) == 0:
        return None

    # Seleccionar la primera (única) detección
    detection = detections[0].tolist()  # Convertir a lista

    # Desempaquetar coordenadas
    x1, y1, x2, y2, conf, cls = detection

    return detection
    #x1, y1, x2, y2, conf, cls

* Función para devolver las coordenadas del objeto drawDetectedDiamonds

In [35]:
def coords(detection):

    if detection is None:
        return None
    x1, y1, x2, y2, conf, cls = detection

    # Filtrar por confianza
    if conf >= 0.55:
      coords = [int(x1), int(y1), int(x2), int(y2)]
      return coords

    return None

## 1.3. Funciones para limpiar el texto

* Se crea una función para filtrar el texto reconocido comprobando que sólo tiene letras y números.

In [7]:
def filter_plate_text(plate_text): #plate_text: Texto detectado por OCR
    #Args: plate_text(Texto detectado por OCR)
    if isinstance(plate_text, list):  # Comprobar si plate_text es una lista
        plate_text = ''.join(plate_text)  # Unir elementos de la lista en una sola cadena
    # Usar una expresión regular para mantener solo letras y números
    return re.sub(r'[^A-Za-z0-9]', '', plate_text)  # Texto filtrado con solo letras y números

* Se crea otra función para filtrar el texto basado en el tamaño de la región.

In [8]:
def filter_text(region, ocr_result, region_threshold):
    #Args: region(Imagen de la región de la placa), ocr_result(Resultado del OCR), region_threshold(Umbral para filtrar regiones de texto por tamaño)
    rectangle_size = region.shape[0]*region.shape[1] #Calcula el área total de la región de la placa
    plate = [] #lista vacía para almacenar el texto filtrado
    print(ocr_result) # Imprime el resultado del OCR para depuración
    for result in ocr_result: # Itera sobre cada resultado del OCR
        length = np.sum(np.subtract(result[0][1], result[0][0])) #Calcula la longitud del texto detectado
        height = np.sum(np.subtract(result[0][2], result[0][1])) #Calcula la altura del texto detectado
        if length*height / rectangle_size > region_threshold: #Comprueba si el área del texto es mayor que el umbral proporcional
            plate.append(result[1]) # Si supera el umbral, añade el texto a la lista
    return plate #Lista de textos filtrados que superan el umbral de tamaño.

## 1.4. Reconocimiento de texto con EasyOCR

In [9]:
def recognize_plate_easyocr(img, coords, reader, region_threshold):
    #Args: img(Imagen original), coords(Coordenadas del cuadro delimitador), reader(Objeto EasyOCR), region_threshold(Umbral mínimo para considerar una región válida)
    xmin, ymin, xmax, ymax = coords
    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)] ## Recortar la región correspondiente a la placa
    ocr_result = reader.readtext(nplate) ## Realizar OCR en la región recortada
    text = filter_text(region=nplate, ocr_result=ocr_result, region_threshold=region_threshold)
    if len(text) == 1:
        text = text[0].upper()
    return text #Texto reconocido en la región



## 1.5. Extracción de la información del vehículo

In [10]:
def vehicle_info(frame, vehicle_detection, plate_detection):
    #Args: frame(Imagen original), vehicle_detection(Detecciones de vehículos), plate_detections(Detecciones de placas)
    vehicle_classes = ['car', 'truck', 'motorcycle'] # Clases de vehículos a considerar
    extracted_info = {} # Lista para almacenar información sobre vehículos detectados

    # Procesar detecciones de vehículos
    if vehicle_detection:
        x1, y1, x2, y2, conf, cls = vehicle_detection
        if conf >= 0.55 and vehicle_model.names[int(cls)] in vehicle_classes:
            extracted_info["vehicle_type"] = vehicle_model.names[int(cls)]

    # Procesar detecciones de placas
    if plate_detection:
        x1, y1, x2, y2, conf, cls = plate_detection # Extraer datos de la detección
        if conf >= 0.55: # Filtrar por confianza mínima de la detección de la placa
            coords = [int(x1), int(y1), int(x2), int(y2)] # Coordenadas del cuadro delimitador
            #Detectar el texto en la placa
            plate_num = recognize_plate_easyocr(img=frame, coords=coords, reader=EASY_OCR, region_threshold=OCR_TH)
            # Filtrar el texto de la placa
            plate_num = filter_plate_text(plate_num)
            # Añadir el número de matrícula a la información a extraer
            extracted_info["plate"] = plate_num

    return extracted_info

## 1.6. Generar fecha ficticia para entrada y salida del vehículo

In [11]:
def generar_fecha_hora():
    ahora = datetime.now()
    delta = timedelta(days=random.randint(-30, 0), hours=random.randint(0, 23), minutes=random.randint(0, 59))
    return ahora + delta

## 1.7. Generar datos aleatorios para cada matrícula

In [27]:
import random

def generar_datos_random(matricula):

    marcas_modelos = {
        "Toyota": ["Supra", "Celica", "Yaris"],
        "Ford": ["GT", "Focus", "Mustang"],
        "Honda": ["NSX", "Civic", "CR-V"],
        "Nissan": ["GTR", "350z", "240sx"],
        "Audi": ["RS7", "R8", "S5"],
        "BMW": ["330e", "M4", "i8"],
        "Mercedes": ["C63S", "GT", "A45"],
        "Tesla": ["3", "Y", "X"],
        "Porsche": ["911", "Cayman", "Taycan"],

    }
    colores = ["Rojo", "Azul", "Negro", "Blanco", "Gris", "Amarillo", "Verde"]
    tarifas = [1, 2, 3]
    parking = [1, 2, 3]

    fecha_entrada = generar_fecha_hora()
    fecha_salida = fecha_entrada + timedelta(seconds=random.randint(0, 100000))

    # Seleccionar datos aleatorios
    marca = random.choice(list(marcas_modelos.keys()))
    modelo = random.choice(marcas_modelos[marca])
    color = random.choice(colores)
    tarifa_id = random.choice(tarifas)
    parking_id = random.choice(parking)

    # Devolver datos generados
    car_info = {
        "matricula_id": matricula['matricula_id'],
        "parking_id": parking_id,
        "fecha_registro_dt": fecha_entrada.strftime("%Y-%m-%d %H:%M:%S"),
        "camara_in_id": fecha_entrada.strftime("%Y-%m-%d %H:%M:%S"),
        "camara_out_id": fecha_salida.strftime("%Y-%m-%d %H:%M:%S"),
        "tarifa_id": tarifa_id,
        "marca_ds": marca,
        "modelo_ds": modelo,
        "color_ds": color,

    }
    return car_info

## 1.8. Insertar los datos de la matrícula en la base de datos

In [32]:
def insertar_matricula(car_info):

    # Inicializar cliente BigQuery
    client = bigquery.Client()

    # Definir la tabla
    table_id = "aiparking-451016.00_aiparking.accesos"

    # Crear el diccionario con los datos
    fila = {
        "matricula_id": car_info["matricula_id"],
        "parking_id": car_info["parking_id"],
        "fecha_registro_dt": car_info["fecha_registro_dt"],
        "tarifa_id": car_info["tarifa_id"],
        "marca_ds": car_info["marca_ds"],
        "modelo_ds": car_info["modelo_ds"],
        "color_ds": car_info["color_ds"],
        "camara_in_id": car_info["camara_in_id"],
        "camara_out_id": car_info["camara_out_id"]
    }

    # Insertar la fila
    errores = client.insert_rows_json(table_id, [fila])

    if errores == []:
        print("Matrícula insertada correctamente.")
    else:
        print("Errores al insertar:", errores)

## 1.9. PIPELINE COMPUTER VISION MODEL

In [14]:
EASY_OCR = easyocr.Reader(['en']) # Inicializar EasyOCR para idioma inglés
OCR_TH = 0.2 # Umbral de confianza para el OCR

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [33]:
def main_CV(img_path=None):

      # Cargamos la imagen
      frame = img_load(img_path)

      # Detección del vehículo con YOLOv5
      vehicle_detection = detectx(frame, model=vehicle_model) #Realiza la detección de vehículo
      print("Vehicle detection:", vehicle_detection)

      # Selección de coordenadas del vehículo
      coords_vehicle = coords(vehicle_detection)

      # Detección de la matrícula con YOLOv5
      plate_detection = detectx(frame, model=plate_model) #Realiza la detección de placas
      print("Plate detection:", plate_detection)

      # Selección de coordenadas del vehículo
      coords_plate = coords(plate_detection)

      # Reconocimiento de texto con EasyOCR
      if coords_plate:
        text = recognize_plate_easyocr(img=frame, coords=coords_plate, reader=EASY_OCR, region_threshold=OCR_TH)
      else:
        text = None

      # Extracción de la información del vehículo
      plate_info = vehicle_info(frame, vehicle_detection, plate_detection)

      # Convertimos formato
      plate = {'matricula_id': plate_info['plate']}
      print("plate")

      # Generar datos aleatorios para cada vehículo
      info = generar_datos_random(plate)

      # Insertar los datos del vehículo en la base de datos
      insertar_matricula(info)

      return print(info)


In [36]:
info2 = main_CV()

Por favor, selecciona una imagen del vehículo:


Saving E1,CAM1,250210085359827,3483JLF,9799.jpg to E1,CAM1,250210085359827,3483JLF,9799 (6).jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Vehicle detection: None


/usr/local/lib/python3.11/dist-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Plate detection: [183.0732879638672, 305.25396728515625, 457.49114990234375, 395.0070495605469, 0.8445020318031311, 0.0]
[([[19, 4], [268, 4], [268, 84], [19, 84]], '3483 JLFI', 0.3905285727810373)]
[([[19, 4], [268, 4], [268, 84], [19, 84]], '3483 JLFI', 0.3905285727810373)]
plate
Matrícula insertada correctamente.
{'matricula_id': '3483JLFI', 'parking_id': 1, 'fecha_registro_dt': '2025-01-25 18:28:46', 'camara_in_id': '2025-01-25 18:28:46', 'camara_out_id': '2025-01-26 14:10:07', 'tarifa_id': 2, 'marca_ds': 'Tesla', 'modelo_ds': 'X', 'color_ds': 'Azul'}


## 1. MODELO LLM

In [37]:
from openai import OpenAI
import json
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client_openai = OpenAI(api_key = api_key)

Enter your OpenAI API Key:··········


### 1.1. Input del usuario

In [38]:
def user_input():
    question = input("Escriba su consulta:")
    return question

### 1.2. Prompt Engineering con ChatGPT

In [47]:
def input_format(question, model="gpt-3.5-turbo", temperature=0):

    prompt_template = """
Eres un experto de SQL para una base de datos de registro de vehiculos. Tu tarea es convertir una pregunta de lenguaje natural a una consulta SQL valida.

## **ESTRUCTURA DE LA BASE DE DATOS**
Nombre del proyecto: 'aiparking-451016'
Nombre de la tabla: '00_aiparking.accesos'

- matricula_id STRING
- parking_id STRING
- fecha_registro_dt DATETIME
- camara_in_id STRING
- camara_out_id STRING
- tipo_vehiculo_id STRING
- marca_ds STRING
- modelo_ds STRING
- color_ds STRING
- tarifa_id STRING

## **TAREA**
Responde UNICAMENTE con la consulta en SQL.

## **EJEMPLOS**
Pregunta: "Cual es el modelo del vehiculo con la matricula 5489 HEB?"
Respuesta: SELECT modelo_ds FROM `aiparking-451016.00_aiparking.accesos` WHERE matricula_id = '5489 HEB';

Pregunta: "Coches de color Gris, Blanco o Azul y que sean de tipo 1"
Respuesta: SELECT * FROM `aiparking-451016.00_aiparking.accesos` WHERE color_ds IN ('Gris', 'Blanco', 'Azul') AND tipo_vehiculo_id = '1';

Pregunta: "A qué hora entró el coche con matrícula 1234ABC el día de ayer?"
Respuesta: SELECT fecha_registro_dt FROM `aiparking-451016.00_aiparking.accesos` WHERE matricula_id = '1234ABC' AND DATE(fecha_registro_dt) = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY);;

## **NUEVA PREGUNTA DEL USUARIO:**
"{question}"

## **RESPUESTA EN SQL:**
"""
    prompt = prompt_template.format(question=question)
    messages = [{"role": "user", "content": prompt}]
    response = client_openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    sql_query = response.choices[0].message.content
    return sql_query


* Ejemplo 1

In [40]:
question_1 = "qué coches con matrícula que acaba en L entraron al parking el día 25/01/2025 entre las 07:00am y las 09:30am?"

In [41]:
response_1 = input_format(question_1)
print(response_1)

SELECT * FROM `aiparking-451016.00_aiparking.vehiculos` 
WHERE matricula_id LIKE '%L' 
AND DATE(fecha_registro_dt) = '2025-01-25' 
AND TIME(fecha_registro_dt) BETWEEN '07:00:00' AND '09:30:00';


### 1.3. Acceso a la Database de BigQuery

In [42]:
from google.colab import auth
auth.authenticate_user()

In [43]:
from google.cloud import bigquery

# Creamos un cliente de BigQuery
client = bigquery.Client(project='aiparking-451016')

In [44]:
import pandas as pd

def sql_ans(query):

      # Consulta SQL
      query_job = client.query(query)

      results = query_job.result()

      # Convertir los resultados a un DataFrame de Pandas
      df = results.to_dataframe()

      return df

### 1.4. Pipeline

In [45]:
def main_LLM(question=None):

    # 1. Pregunta del usuario
    if question is None:
      question = user_input()

    # 2. Conversión lenguaje natural a SQL
    query = input_format(question=question, model="gpt-3.5-turbo", temperature=0)

    # 3. Acceso al Database de BigQuery
    answer = sql_ans(query)

    # 6. Mostrar la salida
    print(answer)

    return answer

In [48]:
main_LLM()

Escriba su consulta:que marca y modelo es el coche 3483JLFI?
   marca_ds modelo_ds
0  Mercedes      C63S
1     Tesla         X


,marca_ds,modelo_ds
0,Mercedes,C63S
1,Tesla,X
